## Variable sharing

>.NET Interactive enables you to write code in multiple languages within a single notebook and in order to take advantage of those languages' different strengths, you might find it useful to share data between them. …You can share variables between .NET subkernels using the `#!share` magic command.
>
> —Jon Sequeira ([@jonsequitur](https://github.com/jonsequitur)), “[Variable sharing](https://github.com/dotnet/interactive/blob/main/docs/variable-sharing.md#variable-sharing)”


In [1]:
#!share --help

Description:
  Get a value from one kernel and create a copy (or a reference if the kernels are in the same process) in another.

Usage:
  #!share <name> [options]

Arguments:
  <Backbone|vscIPyWidgets7|vscIPyWidgets8|__ $YJS$ __>  The name of the value to share. (This is also the default name of the value created in the destination kernel, unless --as is used to specify a different one.)

Options:
  --from <fsharp|javascript|pwsh|value>                The name of the kernel to get the value from.
  --mime-type <application/json|text/html|text/plain>  Share the value as a string formatted to the specified MIME type.
  --as <as>                                            The name to give the the value in the importing kernel.
  -?, -h, --help                                       Show help and usage information





In [2]:
var fromCSharp = "This is from C#";

fromCSharp

This is from C#

In [3]:
#!share --from csharp fromCSharp

let message = $"message: {fromCSharp}"

message

message: This is from C#

### the `#!value` command

>The `#!value` magic command is available to make it as easy as possible to get that text into a variable in your notebook. An important thing to know is that `#!value` is an alias to a subkernel designed just to hold values. This means that once you store something in it, you can access it from another subkernel using `#!share`.


In [4]:
#!value --help

Description:
  Stores a value that can then be shared with other subkernels.

Usage:
  #!value [options]

Options:
  --from-file <from-file>                                       Specifies a file whose contents will be stored.
  --from-url <from-url>                                         Specifies a URL whose content will be stored.
  --from-value <from-value>                                     Specifies a value to be stored directly. Specifying @input:value allows you to prompt the user for this value.
  --mime-type <application/json|text/csv|text/html|text/plain>  A mime type for the value. If specified, displays the value immediately as an output using the specified mime type.
  --name <name> (REQUIRED)                                      The name of the value to create. You can use #!share to retrieve this value from another subkernel.
  -?, -h, --help                                                Show help and usage information




>The simplest way to use `#!value` is to paste some text into the cell. The text will be stored as a string, but unlike using a string literal in C#, F#, or PowerShell, there's no need to escape anything.


In [5]:
#!csharp
#!value --name someJson

{
    "what": "some JSON",
    "why": "to share it with another subkernel"
}

In [6]:
#!csharp
#!share someJson --from value

using System.Text.Json;

var jsonDoc = JsonDocument.Parse(someJson);

jsonDoc.Display("application/json")

`JsonDocument` does not have a `Display()` method. The `Display()` extension method is part of the [formatting](https://github.com/dotnet/interactive/blob/82a2545a4965305f8909e9950ab6e1e1ad4516af/docs/formatting.md) features of .NET Interactive:

>Formatting is invoked when values are displayed either implicitly (using a trailing expression or return statement), using the Display extension method, or using helper methods such as display.
>
> —Don Syme ([@dsymetweets](https://twitter.com/dsymetweets)) and Jon Sequeira ([@jonsequitur](https://github.com/jonsequitur)), “[Formatting](https://github.com/dotnet/interactive/blob/main/docs/formatting.md#formatting)”

There are two other ways to set a `#!value`:

1. `--from-file`
2. `--from-url`


In [7]:
#!value --from-file ./dotnet-interactive-02-file.json --name fileJson

In [8]:
#!share fileJson --from value

jsonDoc = JsonDocument.Parse(fileJson);

jsonDoc.Display("application/json")

In [9]:
#!csharp
#!value --from-url http://jsonplaceholder.typicode.com/users --name wildJson --mime-type application/json

>Regardless of which of these approaches you use, you can additionally choose to display the value in the notebook at the time of submission by using the `--mime-type` option. This accomplishes a few things. If your notebook frontend knows how to display that mime type, you can see it appropriately formatted…


In [10]:
#!share wildJson --from value

jsonDoc = JsonDocument.Parse(wildJson);

jsonDoc.RootElement[0]
    .GetProperty("address")
    .GetProperty("geo")
    .Display("application/json")

Error: (1,30): error CS1503: Argument 1: cannot convert from 'System.Text.Json.JsonDocument' to 'System.ReadOnlyMemory<byte>'

In [11]:
var csharpNow = new { now = DateTime.Now };

>Variable sharing has some limitations to be aware of. When sharing a variable with a subkernel where its compilation requirements aren't met, for example due to a missing using (C#) or open (F#) declaration, a custom type defined in the notebook, or a missing assembly reference, #!share will fail…


In [12]:
#!share --from csharp csharpNow

let now = csharpNow


Error: input.fsx (1,11)-(1,20) typecheck error The value or constructor 'csharpNow' is not defined.

[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼


In [13]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.451701+e231cd6f4a395b6f74568c5ad95a15cbecb85cf2Library version: 1.0.0-beta.23517.1+e231cd6f4a395b6f74568c5ad95a15cbecb85cf2Build date: 2023-10-18T00:15:08.0000000Zhttps://github.com/dotnet/interactive
